## Import & Setup

Note, if you are getting errors like this:
```
...
from .extension import *
  File ".../isaacsim/extsUser/go4robo/go4robo_python/extension.py", line 3, in <module>
    import omni.ext
ModuleNotFoundError: No module named 'omni'
```

Go into `isaacsim/extsUser/go4robo/go4robo_python/__init__.py` and comment out the following line:
```
from .extension import *
```

!! You need to uncomment this line if you plan to run the Isaac Sim extension !!

In [14]:
#Enable reloading because restarting the kernel is a pain
%load_ext autoreload
%autoreload 2

%pip install -q usd-core

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np

from isaacsim.extsUser.go4robo.go4robo_python import bot_3d_problem as g4r_problem
from isaacsim.extsUser.go4robo.go4robo_python import bot_3d_rep as g4r_rep

import json

import os, sys

import copy

from jupyter_dash import JupyterDash
from dash import html, dcc, Input, Output
import plotly.graph_objects as go
import pandas as pd

from tqdm.notebook import trange, tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


2723.94s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Note: you may need to restart the kernel to use updated packages.


## Get the generated designs from csv

In [15]:
df_csv_path = "/Users/robosquiggles/Downloads/designs_Robot_20250429_185333.csv" # This should also contain the prior design at index 0

df = pd.read_csv(df_csv_path)
df

,Index,Name,Cost,Perception Entropy,Generation,s0_type,s0_x,s0_y,s0_z,s0_qw,...,s3_qy,s3_qz,s4_type,s4_x,s4_y,s4_z,s4_qw,s4_qx,s4_qy,s4_qz
0,0,Prior Design,5.0,2.419967,0,2,0.100000,0.200000,0.300000,6.123234e-17,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Design 1,3.0,-0.807904,1,1,1.601489,0.968262,0.313424,2.412117e-01,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Design 2,3.0,3.188194,1,1,0.631031,0.686501,0.834626,7.811300e-01,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Design 3,5.0,2.477862,1,1,0.038734,0.678836,0.211628,7.786989e-02,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Design 4,4.0,-2.829793,1,2,1.071793,0.663795,0.514889,-4.537929e-01,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Design 5,3.0,-0.807904,2,1,1.601489,0.968262,0.313424,2.412117e-01,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,6,Design 6,4.0,-2.829793,2,2,1.071793,0.663795,0.514889,-4.537929e-01,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,7,Design 7,3.0,3.188194,2,1,0.631031,0.686501,0.834626,7.811300e-01,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,8,Design 8,5.0,1.590110,2,1,0.625109,0.678836,0.211628,9.984211e-02,...,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Plot the tradespace

Conveniently you can do this without the Problem definition, assuming you used the standard problem definition.

In [17]:
g4r_problem.plot_tradespace(df)

## Set up the problem from json

In [ ]:
json_prolem_path = "/Users/robosquiggles/Downloads/problem_Robot_20250429_185333.json" # This will contain the prior design at problem.prior_bot

problem_json_dict = json.load(open(json_prolem_path, 'r'))

problem = g4r_problem.SensorPkgOptimization().from_json(problem_json_dict)

prior_bot_from_problem = problem.prior_bot

## Plot any bot from the df above

In [ ]:
INDEX_TO_PLOT = 5

design_dict = df[df["Index"] == INDEX_TO_PLOT].iloc[0].to_dict()
bot = problem.convert_1D_to_bot(design_dict)
selected_fig = bot.plot_bot_3d(
    perception_space=problem.perception_space,
    show=False,
    width=600,
)